# CloudSim-Benchmark 实验结果可视化分析

本工具用于分析和可视化云任务调度算法的实验结果。

## 功能特性
- 📊 多指标性能对比（柱状图）
- 📈 算法稳定性分析（箱线图）
- 🔍 扩展性趋势分析（折线图）
- 📉 性能改进比例计算
- 🎯 自动识别最新实验数据

In [ ]:
# 导入必要的库
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import numpy as np
from pathlib import Path
from IPython.display import display, Markdown

# 基础配置
sns.set_theme(style="whitegrid", font_scale=1.1)
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS']  # 支持中文
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

print("✅ 环境配置完成")

## 1. 配置实验目录

自动搜索最近的实验结果，或手动指定目录。

In [ ]:
def get_latest_exp_dir(mode='batch'):
    """获取最近一次实验的目录"""
    # 尝试多个可能的路径
    possible_paths = [
        os.path.join('..', 'runs', mode),
        os.path.join('runs', mode),
        os.path.join('..', '..', 'runs', mode)
    ]
    
    for base_path in possible_paths:
        if os.path.exists(base_path):
            dirs = [os.path.join(base_path, d) for d in os.listdir(base_path) 
                   if os.path.isdir(os.path.join(base_path, d))]
            if dirs:
                return max(dirs, key=os.path.getmtime)
    return None

# 选择实验模式
MODE = 'batch'  # 可选: 'batch' 或 'realtime'
exp_dir = get_latest_exp_dir(MODE)

if exp_dir:
    print(f"✅ 找到最近的实验目录: {exp_dir}")
    print(f"📁 实验名称: {os.path.basename(exp_dir)}")
else:
    print("❌ 未找到实验结果，请先运行仿真程序。")
    print("提示：运行 ./run batch PSO,WOA,IMPROVED_RL 来生成实验数据")

## 2. 加载实验信息

In [ ]:
if exp_dir:
    info_file = os.path.join(exp_dir, 'experiment_info.txt')
    if os.path.exists(info_file):
        with open(info_file, 'r', encoding='utf-8') as f:
            content = f.read()
            print("=== 实验配置信息 ===")
            print(content)
    else:
        print("⚠ 未找到 experiment_info.txt")
        
    # 列出所有结果文件
    print("\n=== 实验结果文件 ===")
    files = os.listdir(exp_dir)
    for f in files:
        file_path = os.path.join(exp_dir, f)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            print(f"  - {f} ({size} bytes)")

## 3. 加载汇总数据

读取 `summary_avg.csv` 文件，查看各算法的平均性能。

In [ ]:
if exp_dir:
    summary_file = os.path.join(exp_dir, 'summary_avg.csv')
    if os.path.exists(summary_file):
        df_summary = pd.read_csv(summary_file)
        print("✅ 成功加载汇总数据\n")
        display(df_summary)
    else:
        df_summary = None
        print("⚠ 未找到 summary_avg.csv")

## 4. 多指标性能对比 (柱状图)

对比各算法在 Makespan、负载均衡、成本、总时间和适应度等指标上的表现。

In [ ]:
if exp_dir and df_summary is not None:
    # 定义指标映射
    metric_map = {
        'AvgMakespan': '最大完成时间 (Makespan)',
        'AvgLoadBalance': '负载均衡度 (Load Balance)',
        'AvgCost': '总成本 (Cost)',
        'AvgTotalTime': '总执行时间 (Total Time)',
        'AvgFitness': '综合适应度 (Fitness)',
        'AvgWaitingTime': '平均等待时间',
        'AvgResponseTime': '平均响应时间'
    }
    
    # 筛选可用指标
    available_metrics = [m for m in metric_map.keys() if m in df_summary.columns]
    n_metrics = len(available_metrics)
    
    if n_metrics > 0:
        # 动态计算子图布局
        cols = 2
        rows = (n_metrics + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(16, 5 * rows))
        if rows == 1:
            axes = [axes] if cols == 1 else axes
        else:
            axes = axes.flatten()
        
        # 绘制每个指标的柱状图
        for i, metric in enumerate(available_metrics):
            sns.barplot(x='Algorithm', y=metric, data=df_summary, ax=axes[i], palette='viridis')
            axes[i].set_title(metric_map[metric], fontsize=14, fontweight='bold')
            axes[i].set_xlabel('算法', fontsize=12)
            axes[i].set_ylabel('数值', fontsize=12)
            axes[i].tick_params(axis='x', rotation=45)
            
            # 在柱子上方标注数值
            for p in axes[i].patches:
                height = p.get_height()
                if not np.isnan(height):
                    axes[i].annotate(f'{height:.2f}', 
                                   (p.get_x() + p.get_width() / 2., height), 
                                   ha='center', va='bottom', 
                                   xytext=(0, 5), textcoords='offset points',
                                   fontsize=10)
        
        # 隐藏多余的子图
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # 保存图片
        save_path = os.path.join(exp_dir, 'performance_comparison.png')
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"📊 图片已保存至: {save_path}")
    else:
        print("⚠ 没有可用的性能指标")

## 5. 性能改进比例分析

计算各算法相对于基准算法（第一个）的性能提升百分比。

In [ ]:
if exp_dir and df_summary is not None and len(df_summary) > 1:
    # 以第一个算法为基准
    baseline = df_summary.iloc[0]
    print(f"🎯 基准算法: {baseline['Algorithm']}\n")
    
    # 计算改进比例
    improvement_df = df_summary.copy()
    available_metrics = [m for m in metric_map.keys() if m in df_summary.columns]
    
    for m in available_metrics:
        # 对于 Fitness/Cost/LB 等越小越好的指标，改进 = (baseline - current) / baseline
        # 对于其他指标需要具体分析
        improvement_df[f'{m}_提升%'] = ((baseline[m] - improvement_df[m]) / baseline[m] * 100)
    
    improvement_cols = ['Algorithm'] + [col for col in improvement_df.columns if '提升%' in col]
    display(improvement_df[improvement_cols].style.format({col: '{:.2f}%' for col in improvement_cols if col != 'Algorithm'}))

## 6. 算法稳定性分析 (箱线图)

展示各算法在多次独立试验中的性能分布，评估其鲁棒性。

In [ ]:
if exp_dir:
    # 加载所有算法的详细试验数据
    all_trials = []
    for csv_file in glob.glob(os.path.join(exp_dir, '*.csv')):
        filename = os.path.basename(csv_file)
        # 跳过汇总文件
        if filename in ['summary_avg.csv', 'batch_cloudlet_count_summary.csv', 'realtime_cloudlet_count_summary.csv']:
            continue
        
        alg_name = filename.replace('.csv', '').replace('_', '-')
        try:
            df_alg = pd.read_csv(csv_file)
            df_alg['Algorithm'] = alg_name
            all_trials.append(df_alg)
        except Exception as e:
            print(f"⚠ 无法加载 {filename}: {e}")
            continue
    
    if all_trials and len(all_trials) > 0:
        df_combined = pd.concat(all_trials, ignore_index=True)
        
        # 选择主要指标进行箱线图分析
        boxplot_metrics = []
        for col in ['Fitness', 'Makespan', 'LoadBalance', 'Cost']:
            if col in df_combined.columns:
                boxplot_metrics.append(col)
        
        if boxplot_metrics:
            n_box = len(boxplot_metrics)
            fig, axes = plt.subplots(1, min(n_box, 2), figsize=(14, 6))
            if n_box == 1:
                axes = [axes]
            
            for i, metric in enumerate(boxplot_metrics[:2]):
                sns.boxplot(x='Algorithm', y=metric, data=df_combined, ax=axes[i], palette='Set2', width=0.6)
                sns.stripplot(x='Algorithm', y=metric, data=df_combined, ax=axes[i], 
                            color=".25", alpha=0.4, size=5)
                axes[i].set_title(f'{metric} 分布 (Trials)', fontsize=14, fontweight='bold')
                axes[i].set_xlabel('算法')
                axes[i].set_ylabel(metric)
                axes[i].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()
            
            save_path = os.path.join(exp_dir, 'stability_analysis.png')
            fig.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"📊 稳定性分析图已保存至: {save_path}")
        else:
            print("⚠ 未找到可用的指标进行箱线图分析")
    else:
        print("⚠ 未找到详细试验数据，请确保实验的 runs 参数 > 1")

## 7. 任务规模扩展性分析 (折线图)

分析随着任务数量增加，各算法的性能变化趋势（仅适用于 batch-multi 或 realtime-multi 模式）。

In [ ]:
if exp_dir:
    # 检查是否存在批量任务数汇总文件
    multi_summary = None
    for summary_name in ['batch_cloudlet_count_summary.csv', 'realtime_cloudlet_count_summary.csv']:
        path = os.path.join(exp_dir, summary_name)
        if os.path.exists(path):
            multi_summary = path
            break
    
    if multi_summary:
        df_multi = pd.read_csv(multi_summary)
        print(f"✅ 加载批量任务数数据: {os.path.basename(multi_summary)}\n")
        display(df_multi.head(10))
        
        # 绘制扩展性趋势图
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        axes = axes.flatten()
        
        # 指标列表
        scalability_metrics = [
            ('Makespan_Mean', 'Makespan 扩展性'),
            ('Fitness_Mean', 'Fitness 扩展性'),
            ('Cost_Mean', 'Cost 扩展性'),
            ('LoadBalance_Mean', 'Load Balance 扩展性')
        ]
        
        for i, (metric, title) in enumerate(scalability_metrics):
            if metric in df_multi.columns:
                for alg in df_multi['Algorithm'].unique():
                    alg_data = df_multi[df_multi['Algorithm'] == alg]
                    axes[i].plot(alg_data['CloudletCount'], alg_data[metric], 
                               marker='o', linewidth=2.5, label=alg)
                
                axes[i].set_title(title, fontsize=14, fontweight='bold')
                axes[i].set_xlabel('任务数量 (Cloudlet Count)', fontsize=12)
                axes[i].set_ylabel(metric.replace('_Mean', ''), fontsize=12)
                axes[i].legend(title='算法', loc='best')
                axes[i].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        save_path = os.path.join(exp_dir, 'scalability_analysis.png')
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"📊 扩展性分析图已保存至: {save_path}")
    else:
        print("⚠ 未找到批量任务数汇总文件")
        print("提示: 运行 './run batch-multi 50,100,200,500' 来生成扩展性实验数据")

## 8. 算法综合排名

In [ ]:
if exp_dir and df_summary is not None:
    if 'AvgFitness' in df_summary.columns:
        # 按 Fitness 排序（越小越好）
        df_ranked = df_summary.sort_values('AvgFitness').reset_index(drop=True)
        df_ranked.index = df_ranked.index + 1
        df_ranked.index.name = '排名'
        
        print("🏆 算法综合性能排名 (基于 Fitness):\n")
        display(df_ranked[['Algorithm', 'AvgFitness', 'AvgMakespan', 'AvgCost', 'AvgLoadBalance']])

---

## 使用说明

### 1. 运行实验
```bash
# 批处理模式对比实验（运行2次取平均）
./run batch PSO,WOA,IMPROVED_RL 42 2

# 批量任务数实验
./run batch-multi 50,100,200,500 3 PSO,IMPROVED_RL
```

### 2. 打开本 Notebook
```bash
cd draw
jupyter notebook visualize_results.ipynb
```

### 3. 点击 "Run All" 即可生成所有图表

### 4. 图表说明
- **柱状图**: 显示各算法在不同指标上的平均值对比
- **箱线图**: 显示算法在多次运行中的稳定性（方差越小越稳定）
- **折线图**: 显示随任务规模增长的性能趋势